In [1]:
# ! pip install pandas web3 hexbytes rlp fastlz clickhouse-connect
# ! pip install python-dotenv

### Readme
FastLZ needs Python version 3.9x or lower, make sure your environment is using a later python version

In [2]:
import pandas as pd
from web3 import Web3
from hexbytes import HexBytes
import ast
import rlp
from rlp.sedes import Binary, big_endian_int, binary, List
from eth_utils import to_bytes, to_hex
import fastlz
import sys
import os
import dotenv
dotenv.load_dotenv()
sys.path.append("../../helper_functions")
import clickhouse_utils as ch
sys.path.pop()

client = ch.connect_to_clickhouse_db() #Default is OPLabs DB

In [3]:
#Configuration
chain_mappings_list = [
        {'schema_name': 'op', 'display_name': 'OP Mainnet', 'chain_id': 10},
        # {'schema_name': 'base', 'display_name': 'Base'},
        # {'schema_name': 'mode', 'display_name': 'Mode'},
        # {'schema_name': 'fraxtal', 'display_name': 'Fraxtal'},
    # Add more mappings as needed
]

days_of_data = 7
# chain_mappings_dict = {item['schema_name']: item['display_name'] for item in chain_mappings_list}


In [4]:
# Test transaction receipt
from web3 import Web3
op_rpc = os.getenv("OP_PUBLIC_RPC")
w3 = Web3(Web3.HTTPProvider(op_rpc))

tx_test = '0xcea81f2e836a37b38ba82afd37e6f66c02e348e7b89538aa232013d91edcb926'
tx = w3.eth.get_transaction(tx_test)
txr = w3.eth.get_transaction_receipt(tx_test)
# # txraw = w3.eth.get_raw_transaction(tx_test)
print(tx)
# print(txr)
# # print(txraw)

AttributeDict({'blockHash': HexBytes('0x694fb0156a5719a8ee75ebf2568e8a8d55899db9312127f0d4fe5001c06ac397'), 'blockNumber': 120731426, 'chainId': 10, 'from': '0x899e837095a0F3CC62FB05998559Df90F26A1F46', 'gas': 69534, 'gasPrice': 4307539, 'hash': HexBytes('0xcea81f2e836a37b38ba82afd37e6f66c02e348e7b89538aa232013d91edcb926'), 'input': HexBytes('0xa9059cbb0000000000000000000000007a6e883eec3dd33528115637ea01b3b64e2f58490000000000000000000000000000000000000000000000009e34ef99a7740000'), 'nonce': 206, 'r': HexBytes('0x6727a53c0972c55923242cea052dc4e1105d7b65c91c442e2741440965eac357'), 's': HexBytes('0x0a8e71aea623adb7b5562fb9a779634f3b84dad7be1e1f22caaa640db352a6ff'), 'to': '0xdC6fF44d5d932Cbd77B52E5612Ba0529DC6226F1', 'transactionIndex': 4, 'type': 0, 'v': 55, 'value': 0})


In [5]:
# may not sufficent due to missing transaction signature fields

# Get L2 Txs from Clickhouse / Goldsky
query = '''
        SELECT @chain_id@ as chain_id, nonce, gas, max_fee_per_gas, max_priority_fee_per_gas,
                to_address as to, value, input, block_number, hash, receipt_gas_used
        FROM @chain_db_name@_transactions
        WHERE gas_price > 0
        AND block_timestamp < DATE_TRUNC('day',NOW())
        AND block_timestamp >= DATE_TRUNC('day',NOW() - interval '@num_days@ days')
        AND hash = lower('0xcea81f2e836a37b38ba82afd37e6f66c02e348e7b89538aa232013d91edcb926')
        and block_number = 120731426
        LIMIT 5

        SETTINGS max_execution_time = 3000
'''
dfs = []
for chain in chain_mappings_list:
        query_map = query

        query_map = query_map.replace("@chain_db_name@", chain['schema_name'])
        query_map = query_map.replace("@chain_id@", str(chain['chain_id']))
        query_map = query_map.replace("@num_days@", str(days_of_data))

        result_df = client.query_df(query_map)
        dfs.append(result_df)

txs_df = pd.concat(dfs)

# Add Dummy Signature and fields
txs_df['access_list'] = '[]'
txs_df['access_list'] = txs_df['access_list'].apply(ast.literal_eval)
txs_df['r'] = '0xf267a8b78655826edfcb5cd4b852804c0ae1e1ddb14021b79a117a184f87678a'
txs_df['s'] = '0x5693e2fcdf1280fc8e957e900dece6edd9bcd201a233d3f1edb58d64d54e7fa9'
txs_df['v'] = '56'

# txs_df

In [6]:
display(txs_df.transpose())

,0
chain_id,10
nonce,206
gas,69534
max_fee_per_gas,0
max_priority_fee_per_gas,0
to,b'0xdc6ff44d5d932cbd77b52e5612ba0529dc6226f1'
value,0
input,0xa9059cbb0000000000000000000000007a6e883eec3d...
block_number,120731426
hash,b'0xcea81f2e836a37b38ba82afd37e6f66c02e348e7b8...


In [8]:
# Process transactions and RLP encode
#https://ethereum.org/en/developers/docs/transactions/

def process_and_encode_transaction(row):

    tx_params = {
        'chainId': row['chain_id'],
        'nonce': row['nonce'],
        'maxPriorityFeePerGas': row['max_priority_fee_per_gas'],
        'maxFeePerGas': row['max_fee_per_gas'],
        'gas': row['gas'],
        'to': to_bytes(hexstr=row["to"].decode('utf-8')),
        'value': to_bytes(row['value']),
        'input': HexBytes(row['input']),
        
        'accessList': row['access_list'],  # This is assumed to be an empty list or properly formatted
        'v': row['v'],
        'r': HexBytes(row['r']),
        's': HexBytes(row['s'])
    }
    
    print(tx_params)  # Debug: Print intermediate values
            #EIP1559Transaction(
    transaction = [ 
        tx_params['chainId'],
        tx_params['nonce'],
        tx_params['gas'],
        tx_params['maxFeePerGas'],
        tx_params['maxPriorityFeePerGas'],
        tx_params['to'],
        tx_params['value'],
        tx_params['input'],
        tx_params['accessList'],
        tx_params['v'],
        tx_params['r'],
        tx_params['s']
    ]

    encoded_tx = rlp.encode(transaction)
    return Web3.to_hex(encoded_tx), len(encoded_tx)

txs_df[['encoded_transaction', 'len_encoded_transaction']] = txs_df.apply(process_and_encode_transaction, axis=1, result_type='expand')
txs_df

# print(txs_df['encoded_transaction'][0])
# print(len(txs_df['encoded_transaction'][0]))

{'chainId': 10, 'nonce': 206, 'maxPriorityFeePerGas': 0, 'maxFeePerGas': 0, 'gas': 69534, 'to': b'\xdco\xf4M]\x93,\xbdw\xb5.V\x12\xba\x05)\xdcb&\xf1', 'value': b'\x00', 'input': HexBytes('0xa9059cbb0000000000000000000000007a6e883eec3dd33528115637ea01b3b64e2f58490000000000000000000000000000000000000000000000009e34ef99a7740000'), 'accessList': [], 'v': '56', 'r': HexBytes('0xf267a8b78655826edfcb5cd4b852804c0ae1e1ddb14021b79a117a184f87678a'), 's': HexBytes('0x5693e2fcdf1280fc8e957e900dece6edd9bcd201a233d3f1edb58d64d54e7fa9')}


,chain_id,nonce,gas,max_fee_per_gas,max_priority_fee_per_gas,to,value,input,block_number,hash,receipt_gas_used,access_list,r,s,v,encoded_transaction,len_encoded_transaction
0,10,206,69534,0,0,b'0xdc6ff44d5d932cbd77b52e5612ba0529dc6226f1',0,0xa9059cbb0000000000000000000000007a6e883eec3d...,120731426,b'0xcea81f2e836a37b38ba82afd37e6f66c02e348e7b8...,34408,[],0xf267a8b78655826edfcb5cd4b852804c0ae1e1ddb140...,0x5693e2fcdf1280fc8e957e900dece6edd9bcd201a233...,56,0xf8ab0a81ce83010f9e808094dc6ff44d5d932cbd77b5...,173


In [10]:
# Function to compress transaction data
def compress_transaction(encoded_transaction):

    hex_string = encoded_transaction[2:]
    # Convert the hexadecimal string to bytes
    byte_string = bytes.fromhex(hex_string)
    compressed_data = fastlz.compress(byte_string)

    return compressed_data.hex(), len(compressed_data)
# Define a function to apply to each row of the DataFrame
def process_and_compress_transaction(row):
    encoded_tx = row['encoded_transaction']
    compressed_tx, len_tx = compress_transaction(encoded_tx)
    return compressed_tx, len_tx
# Apply compression to each transaction in the DataFrame
txs_df[['compressed_transaction', 'compressed_transaction_length']] = txs_df.apply(process_and_compress_transaction, axis=1, result_type='expand')

txs_df

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_19553/4225734468.py:7: DeprecationWarning: PY_SSIZE_T_CLEAN will be required for '#' formats
  compressed_data = fastlz.compress(byte_string)


,chain_id,nonce,gas,max_fee_per_gas,max_priority_fee_per_gas,to,value,input,block_number,hash,receipt_gas_used,access_list,r,s,v,encoded_transaction,len_encoded_transaction,compressed_transaction,compressed_transaction_length
0,10,206,69534,0,0,b'0xdc6ff44d5d932cbd77b52e5612ba0529dc6226f1',0,0xa9059cbb0000000000000000000000007a6e883eec3d...,120731426,b'0xcea81f2e836a37b38ba82afd37e6f66c02e348e7b8...,34408,[],0xf267a8b78655826edfcb5cd4b852804c0ae1e1ddb140...,0x5693e2fcdf1280fc8e957e900dece6edd9bcd201a233...,56,0xf8ab0a81ce83010f9e808094dc6ff44d5d932cbd77b5...,173,ad0000001ff8ab0a81ce83010f9e808094dc6ff44d5d93...,157


In [11]:
# Aggregate L2

In [12]:
# Pull aggregate L1 data

In [13]:
# Generate L2 : L1 ratio metrics